In [43]:
# ! pip install transformers

# ! pip install torch
# ! pip install scikit-image


import torch
from torch import nn
from transformers import SegformerImageProcessor, SegformerForSemanticSegmentation
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import requests
import os
import glob
from skimage import color
import pickle
import csv
import cv2 as cv
import math
import pandas as pd
os.chdir('/home/ubuntu/Beauty_Filter_Resubmission')
# os.chdir('/home/ubuntu/CategoricalvContinuous/multidimensional_skin_analysis')

In [44]:


#Original
original = "data/original/original_val_train"
#Filters
baby_cute_face="data/filters/baby_cute_face_combined"
baby_girl="data/filters/babygirl_combined"
big_city_life="data/filters/big_city_life_combined"
blochi_gora="data/filters/blochigora_combined"
caramel_macchiato="data/filters/caramel_macchiato_combined"
cute_baby_face="data/filters/cute_baby_face_combined"
pretty="data/filters/pretty_combined"
shiny_foxy = "data/filters/shiny_foxy_combined"
snatched = "data/filters/snatched_combined"

#Test
test_baby_cute_face="data/test/5_images_baby_cute_face"
test_baby_girl="data/test/5_images_babygirl"
test_big_city_life="data/test/5_images_big_city_life"
test_blochi_gora="data/test/5_images_blochigora"
test_caramel_macchiato="data/test/5_images_caramel_macchiato"
test_cute_baby_face="data/test/5_images_cute_baby_face"
test_pretty="data/test/5_images_pretty"
test_shiny_foxy = "data/test/5_images_shiny_foxy"
test_snatched = "data/test/5_images_snatched"






In [45]:
# Get images from original directory
original_images = glob.glob(os.path.join(original, "*.jpg"))
original_basenames = set(os.path.basename(img) for img in original_images)

print(f'There are {len(original_images)} images in the original data.')
print(f'Original basenames: {len(original_basenames)}')

# Get images from all filter directories
baby_cute_face_images = glob.glob(os.path.join(baby_cute_face, "*.jpg"))
baby_girl_images = glob.glob(os.path.join(baby_girl, "*.jpg"))
big_city_life_images = glob.glob(os.path.join(big_city_life, "*.jpg"))
blochi_gora_images = glob.glob(os.path.join(blochi_gora, "*.jpg"))
caramel_macchiato_images = glob.glob(os.path.join(caramel_macchiato, "*.jpg"))
cute_baby_face_images = glob.glob(os.path.join(cute_baby_face, "*.jpg"))
pretty_images = glob.glob(os.path.join(pretty, "*.jpg"))
shiny_foxy_images = glob.glob(os.path.join(shiny_foxy, "*.jpg"))
snatched_images = glob.glob(os.path.join(snatched, "*.jpg"))

# Get basenames for each filter
baby_cute_face_basenames = set(os.path.basename(img) for img in baby_cute_face_images)
baby_girl_basenames = set(os.path.basename(img) for img in baby_girl_images)
big_city_life_basenames = set(os.path.basename(img) for img in big_city_life_images)
blochi_gora_basenames = set(os.path.basename(img) for img in blochi_gora_images)
caramel_macchiato_basenames = set(os.path.basename(img) for img in caramel_macchiato_images)
cute_baby_face_basenames = set(os.path.basename(img) for img in cute_baby_face_images)
pretty_basenames = set(os.path.basename(img) for img in pretty_images)
shiny_foxy_basenames = set(os.path.basename(img) for img in shiny_foxy_images)
snatched_basenames = set(os.path.basename(img) for img in snatched_images)

# Print counts for each filter
print(f'Baby cute face: {len(baby_cute_face_basenames)}')
print(f'Baby girl: {len(baby_girl_basenames)}')
print(f'Big city life: {len(big_city_life_basenames)}')
print(f'Blochi gora: {len(blochi_gora_basenames)}')
print(f'Caramel macchiato: {len(caramel_macchiato_basenames)}')
print(f'Cute baby face: {len(cute_baby_face_basenames)}')
print(f'Pretty: {len(pretty_basenames)}')
print(f'Shiny foxy: {len(shiny_foxy_basenames)}')
print(f'Snatched: {len(snatched_basenames)}')

# Find common images across ALL directories (original + all filters)
common_images = original_basenames.intersection(
    baby_cute_face_basenames,
    baby_girl_basenames,
    big_city_life_basenames,
    blochi_gora_basenames,
    caramel_macchiato_basenames,
    cute_baby_face_basenames,
    pretty_basenames,
    shiny_foxy_basenames,
    snatched_basenames
)

print(f'\nCommon images across all directories: {len(common_images)}')

# If you want common images just between filters (excluding original)
common_filter_images = baby_cute_face_basenames.intersection(
    baby_girl_basenames,
    big_city_life_basenames,
    blochi_gora_basenames,
    caramel_macchiato_basenames,
    cute_baby_face_basenames,
    pretty_basenames,
    shiny_foxy_basenames,
    snatched_basenames
)

print(f'Common images across all filters (no original): {len(common_filter_images)}')

There are 12644 images in the original data.
Original basenames: 12644
Baby cute face: 12647
Baby girl: 12647
Big city life: 12647
Blochi gora: 12647
Caramel macchiato: 12647
Cute baby face: 12647
Pretty: 12647
Shiny foxy: 12647
Snatched: 12647

Common images across all directories: 12643
Common images across all filters (no original): 12646


In [46]:
# Test
import os
import glob

# Test directories
test_baby_cute_face = "data/test/5_images_baby_cute_face"
test_baby_girl = "data/test/5_images_babygirl"
test_big_city_life = "data/test/5_images_big_city_life"
test_blochi_gora = "data/test/5_images_blochigora"
test_caramel_macchiato = "data/test/5_images_caramel_macchiato"
test_cute_baby_face = "data/test/5_images_cute_baby_face"
test_pretty = "data/test/5_images_pretty"
test_shiny_foxy = "data/test/5_images_shiny_foxy"
test_snatched = "data/test/5_images_snatched"

# Get images from all test filter directories
test_baby_cute_face_images = glob.glob(os.path.join(test_baby_cute_face, "*.jpg"))
test_baby_girl_images = glob.glob(os.path.join(test_baby_girl, "*.jpg"))
test_big_city_life_images = glob.glob(os.path.join(test_big_city_life, "*.jpg"))
test_blochi_gora_images = glob.glob(os.path.join(test_blochi_gora, "*.jpg"))
test_caramel_macchiato_images = glob.glob(os.path.join(test_caramel_macchiato, "*.jpg"))
test_cute_baby_face_images = glob.glob(os.path.join(test_cute_baby_face, "*.jpg"))
test_pretty_images = glob.glob(os.path.join(test_pretty, "*.jpg"))
test_shiny_foxy_images = glob.glob(os.path.join(test_shiny_foxy, "*.jpg"))
test_snatched_images = glob.glob(os.path.join(test_snatched, "*.jpg"))

# Get basenames for each test filter
test_baby_cute_face_basenames = set(os.path.basename(img) for img in test_baby_cute_face_images)
test_baby_girl_basenames = set(os.path.basename(img) for img in test_baby_girl_images)
test_big_city_life_basenames = set(os.path.basename(img) for img in test_big_city_life_images)
test_blochi_gora_basenames = set(os.path.basename(img) for img in test_blochi_gora_images)
test_caramel_macchiato_basenames = set(os.path.basename(img) for img in test_caramel_macchiato_images)
test_cute_baby_face_basenames = set(os.path.basename(img) for img in test_cute_baby_face_images)
test_pretty_basenames = set(os.path.basename(img) for img in test_pretty_images)
test_shiny_foxy_basenames = set(os.path.basename(img) for img in test_shiny_foxy_images)
test_snatched_basenames = set(os.path.basename(img) for img in test_snatched_images)

# Print counts for each test filter
print(f'Test Baby cute face: {len(test_baby_cute_face_basenames)}')
print(f'Test Baby girl: {len(test_baby_girl_basenames)}')
print(f'Test Big city life: {len(test_big_city_life_basenames)}')
print(f'Test Blochi gora: {len(test_blochi_gora_basenames)}')
print(f'Test Caramel macchiato: {len(test_caramel_macchiato_basenames)}')
print(f'Test Cute baby face: {len(test_cute_baby_face_basenames)}')
print(f'Test Pretty: {len(test_pretty_basenames)}')
print(f'Test Shiny foxy: {len(test_shiny_foxy_basenames)}')
print(f'Test Snatched: {len(test_snatched_basenames)}')

# Find common images across ALL test directories
test_common_images = test_baby_cute_face_basenames.intersection(
    test_baby_girl_basenames,
    test_big_city_life_basenames,
    test_blochi_gora_basenames,
    test_caramel_macchiato_basenames,
    test_cute_baby_face_basenames,
    test_pretty_basenames,
    test_shiny_foxy_basenames,
    test_snatched_basenames
)

print(f'\nCommon test images across all directories: {len(test_common_images)}')

Test Baby cute face: 5
Test Baby girl: 5
Test Big city life: 5
Test Blochi gora: 5
Test Caramel macchiato: 5
Test Cute baby face: 5
Test Pretty: 5
Test Shiny foxy: 5
Test Snatched: 5

Common test images across all directories: 5


In [47]:
import os
import pickle

def save_to_pkl(filename, save_directory):
    # Ensure the directory exists
    os.makedirs(save_directory, exist_ok=True)

    # Save common images to a pkl file in the specified directory
    with open(os.path.join(save_directory, 'common_images.pkl'), 'wb') as f:
        pickle.dump(filename, f)

# Save the common images to a pickle file if needed
if len(common_images) > 0:
    save_directory = 'data/common_images'
    save_to_pkl(common_images, save_directory)
    print(f"Saved {len(common_images)} common images to {save_directory}/common_images.pkl")
else:
    print("No common images found to save")

Saved 12643 common images to data/common_images/common_images.pkl


In [48]:
# # Only use color images
# def is_greyscale(img_path):
#     img = Image.open(img_path).convert('RGB')
#     np_img = np.array(img)
#     return np.all(np_img[..., 0] == np_img[..., 1]) and np.all(np_img[..., 0] == np_img[..., 2])


# full_paths = [os.path.join(original, img) for img in random_images]

# rgb_images = [img for img in full_paths if not is_greyscale(img)]

# print(f'There are {len(rgb_images)} color images in the directory.')

In [49]:
#Function to display original and filtered images side by side
def display_images(data, original, filtered):
    for img_name in data:
        # Construct the full path to the original and filtered image
        original_img_path = os.path.join(original, img_name)
        filter_img_path = os.path.join(filtered, img_name)

        # Read the images
        original_img = mpimg.imread(original_img_path)
        print(original_img_path)
        filter_img = mpimg.imread(filter_img_path)
        #Flip the filtered images
        # filter_img = np.fliplr(filter_img)

        print(filter_img_path)


        plt.figure()

        # Display the original image
        plt.subplot(1, 2, 1)
        plt.imshow(original_img)
        plt.axis('off')  # to hide the axis
        plt.title('Original')

        # Display the filtered image
        plt.subplot(1, 2, 2)
        plt.imshow(filter_img)
        plt.axis('off')  # to hide the axis
        plt.title('Filtered')

        plt.show()

In [50]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# import numpy as np

# # Get just the base names of the images without directory path
# image_names = [os.path.basename(img) for img in common_images]

# # Pass these to the display_images function
# display_images(image_names, original, filtered)




In [51]:
# %pip install torch --upgrade

# import skin segmentation model
image_processor = SegformerImageProcessor.from_pretrained("jonathandinu/face-parsing")
model = SegformerForSemanticSegmentation.from_pretrained("jonathandinu/face-parsing")

In [52]:
#This function processes the image, runs it through the model to get predictions,
# and then extracts and visualizes the skin regions from the image. 
def run_inference(image, image_processor, model):
    # Run inference on the image
    
    # Process the image using the image_processor to get the inputs in the required format (PyTorch tensors)
    inputs = image_processor(images=image, return_tensors="pt")
    
    # Pass the processed inputs through the model to get the outputs
    outputs = model(**inputs)
    
    # Extract the logits (raw predictions) from the model outputs
    logits = outputs.logits
    
    # Upsample the logits to match the original image size using bilinear interpolation
    upsampled_logits = nn.functional.interpolate(logits, size=image.size[::-1], mode='bilinear', align_corners=False)
    
    # Get the labels by finding the index of the maximum logit value along the channel dimension
    labels = upsampled_logits.argmax(dim=1)[0]
    
    # Convert the labels tensor to a NumPy array for further processing
    labels_viz = labels.cpu().numpy()

    # Extract the skin from the image by setting non-skin labels to 0
    labels_viz[labels_viz > 2] = 0  # Set labels greater than 2 to 0
    labels_viz[labels_viz == 0] = 0  # Ensure label 0 remains 0
    labels_viz[labels_viz == 1] = 1  # Set label 1 to 1 (indicating skin)

    # Return the processed labels visualization
    return labels_viz

In [53]:
def display_mask(random_images, test_original, test_filter, image_processor, model):
    for img_name in random_images:
        # Construct the full path to the original and filtered image
        original_img_path = os.path.join(test_original, img_name)
        filter_img_path = os.path.join(test_filter, img_name)

        # Read the images
        original_img = Image.open(original_img_path)
        filter_img = Image.open(filter_img_path)
        #Flip filtered image
        # filter_img = np.array(filter_img)
        # filter_img = np.fliplr(filter_img)
        # filter_img = Image.fromarray(filter_img)


        # Run inference on the original and filtered images
        labels_viz_original = run_inference(original_img, image_processor, model)
        labels_viz_filter = run_inference(filter_img, image_processor, model)


        plt.figure()

        # Display the skin from the original image
        plt.subplot(1, 2, 1)
        plt.imshow(labels_viz_original)
        plt.axis('off')  # to hide the axis
        plt.title('Original')

        # Display the skin from the filtered image
        plt.subplot(1, 2, 2)
        plt.imshow(labels_viz_filter)
        plt.axis('off')  # to hide the axis
        plt.title('Filtered')

        plt.show()





In [54]:
# display_mask(rgb_images, original, filtered, image_processor, model)

In [55]:

def apply_mask(image, mask):
    # Convert the image and mask to numpy arrays
    image_array = np.array(image)
    mask_array = np.array(mask)

    #mask is a boolean array
    mask_array = mask_array.astype(bool)

    # Apply  mask to the image
    masked_image_array = np.where(mask_array[..., None], image_array, 0)

    # Convert mask image array back to a PIL Image
    masked_image = Image.fromarray(masked_image_array)

    return masked_image
def display_skin(random_images, test_original, test_filter, image_processor, model):
    for img_name in random_images:
        # Construct the full path to the original and filtered image
        original_img_path = os.path.join(test_original, img_name)
        filter_img_path = os.path.join(test_filter, img_name)

        # Read the images
        original_img = Image.open(original_img_path)
        filter_img = Image.open(filter_img_path)
        # #Flip filtered image
        # filter_img = np.array(filter_img)
        # filter_img = np.fliplr(filter_img)
        # filter_img = Image.fromarray(filter_img)

        # Run inference on the original and filtered images
        labels_viz_original = run_inference(original_img, image_processor, model)
        labels_viz_filter = run_inference(filter_img, image_processor, model)

        # Apply the masks to the original and filtered images
        original_img_with_mask = apply_mask(original_img, labels_viz_original)
        filter_img_with_mask = apply_mask(filter_img, labels_viz_filter)
        print(original_img_with_mask, filter_img_with_mask)

        plt.figure()

        # Display the skin from the original image
        plt.subplot(1, 2, 1)
        plt.imshow(original_img_with_mask)
        plt.axis('off')  # to hide the axis
        plt.title('Original')

        # Display the skin from the filtered image
        plt.subplot(1, 2, 2)
        plt.imshow(filter_img_with_mask)
        plt.axis('off')  # to hide the axis
        plt.title('Filtered')

        plt.show()

In [56]:
# display_skin(common_images, original, filtered, image_processor, model)

In [57]:
def run_inference(image, image_processor, model):
    # Run inference on the image
    inputs = image_processor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits
    upsampled_logits = nn.functional.interpolate(logits, size=image.size[::-1], mode='bilinear', align_corners=False)
    labels = upsampled_logits.argmax(dim=1)[0]
    labels_viz = labels.cpu().numpy()

    #Nose
    labels_viz[labels_viz ==2] = 1
    #Neck
    labels_viz[labels_viz ==17] = 1
    #Ears
    labels_viz[labels_viz ==8] = 1
    labels_viz[labels_viz ==9] = 1
    #Non-Skin
    labels_viz[labels_viz >2] = 0

    return labels_viz

In [58]:

def get_ita(image, mask):
    image = np.array(image)  # Convert PIL image to NumPy array
    mask = np.array(mask)   
    # print(f"Image shape: {image.shape}")
    # print(f"Mask shape: {mask.shape}")
     # Convert mask to 2D if it is 3D
    if mask.ndim == 3:
        mask = mask[:, :, 0]
    assert image.shape[:2] == mask.shape

   
    # Split image channels info
    c1, c2, c3 = cv.split(image)
    color_feature = []

    # Only process mask != 0 channel region
    c1 = c1[np.where(mask != 0)]
    c2 = c2[np.where(mask != 0)]
    c3 = c3[np.where(mask != 0)]

    # Extract mean
    mean_1 = np.mean(c1)
    mean_2 = np.mean(c2)
    mean_3 = np.mean(c3)

    color_feature.extend([mean_1, mean_2, mean_3])

    # Rescale skin tone
    tone_rgb = [[[x/255 for x in color_feature]]]
    # plt.imshow(image)
    # plt.show()
    # plt.imshow(tone_rgb)
    # plt.show()

    # Convert to LAB color space
    tone_lab = color.rgb2lab(tone_rgb)

    # Get individual colors in LAB color space
    l, a, b = tone_lab.flatten()

    # Calculate ita angle
    ita = np.arctan((l-50)/b)*(180/math.pi)
    #print(ita)

    return ita

In [82]:

#Define the 10 official Monk Skin Tone (MST) color swatches using their hex values.
MST_HEX = [
"#f6ede4", "#f3e7db", "#f7ead0", "#eadaba", "#d7bd96",
"#a07e56", "#825c43", "#604134", "#3a312a", "#292420"
]

# helper function  to convert hex → normalized RGB.
def _hex_to_rgb01(hexstr):
    # Remove # if present.
    hexstr = hexstr.lstrip('#')
    # Convert hex pairs to base-10 integers, then scale to [0,1].
    r = int(hexstr[0:2], 16) / 255.0
    g = int(hexstr[2:4], 16) / 255.0
    b = int(hexstr[4:6], 16) / 255.0
    # Return RGB triplet as float32.
    return np.array([r, g, b], dtype=np.float32)

# Precompute MST swatches in Lab
# Convert all MST hex swatches → RGB → Lab color space.
# Reshape to a list of 10 reference Lab vectors.
MST_LAB = color.rgb2lab(np.array([_hex_to_rgb01(h) for h in MST_HEX])[None, :, :]).reshape(-1,3)

# Computes the robust average skin color in Lab space
def masked_skin_lab_stats(pil_image, mask_bool, drop_pct=0.10):
    """Return robust (trimmed-mean) L*a*b* of skin pixels."""
    
    
    # Convert image to normalized float32 NumPy array.
    img = np.asarray(pil_image).astype(np.float32) / 255.0
    
    # Ensure mask is 2-D (strip redundant channels from segmentation map).
    if mask_bool.ndim == 3:  # (H,W,3) -> (H,W)
        mask_bool = mask_bool[...,0] != 0
    
    # Extract only pixels marked as “skin.”
    skin = img[mask_bool]

    # Return None if no skin present.
    if skin.size == 0:
        return None  # no skin found
    # RGB -> Lab
    # Convert all skin pixels from RGB → Lab.
    skin_lab = color.rgb2lab(skin.reshape(1, -1, 3)).reshape(-1, 3)
    # Trim extremes in L* to reduce highlights/shadows
    L = skin_lab[:,0]
    lo, hi = np.quantile(L, [drop_pct, 1-drop_pct])
    keep = (L >= lo) & (L <= hi)
    skin_lab = skin_lab[keep]
   
    # Robust average (median works well too)
    # skin_lab
    # return np.mean(skin_lab, axis=0)
    return np.median(skin_lab, axis=0)
# Compute ΔE between one Lab vector and a list of reference Lab colors.
def deltaE_ciede2000_batch(lab, refs):
    # skimage has deltaE_ciede2000, but vectorized here for speed via skimage
    from skimage.color import deltaE_ciede2000
    # Compute ΔE between the sample and each swatch.
    # Returns an array of 10 distances.
    
    return np.array([deltaE_ciede2000(lab[None,:], r[None,:])[0] for r in refs])

# # Assign the MST bin to an image using its skin pixels.
# def assign_mst_bin(pil_image, mask_bool):
#     # Get robust median Lab value for the skin.
#     lab = masked_skin_lab_stats(pil_image, mask_bool)
    
#     # Return no result if no skin found.
#     if lab is None:
#         return None, None, None
#     # Compute ΔE to each of the 10 MST swatches.
#     dE = deltaE_ciede2000_batch(lab, MST_LAB)
#     # Pick the swatch with minimum ΔE.
#     bin_idx = int(np.argmin(dE))  # 0..9
#     # simple confidence: gap between best and second-best
#     # Compute confidence = difference between best and second-best ΔE.
#     order = np.argsort(dE)
#     conf = float(dE[order[1]] - dE[order[0]])
#     return bin_idx+1, float(dE[bin_idx]), conf  # 1..10, distance, confidence gap


def assign_mst_bin(pil_image, mask_bool, tau=5.0):
    """
    Assign an MST bin using ΔE to all 10 swatches and compute a
    probability-like confidence score based on a softmax over -ΔE.

    Args:
        pil_image: PIL.Image, RGB face image.
        mask_bool: boolean mask (H,W) where True = skin pixel.
        tau: temperature parameter controlling sharpness of confidence
             (smaller tau -> sharper probabilities, larger tau -> softer).

    Returns:
        mst_bin (int or None): 1..10 (light to dark), or None if no skin.
        best_deltaE (float or None): ΔE for the chosen bin.
        confidence (float or None): softmax probability of chosen bin in [0,1].
    """
    # 1. Get median Lab color of skin
    lab = masked_skin_lab_stats(pil_image, mask_bool)
    if lab is None:
        return None, None, None

    # 2. ΔE to each MST swatch (length 10)
    dE = deltaE_ciede2000_batch(lab, MST_LAB)  # shape: (10,)

    # 3. Convert distances to "closeness" scores using softmax over -ΔE
    #    - subtract min(dE) for numerical stability
    dE = np.asarray(dE, dtype=np.float32)
    dE_shifted = dE - dE.min()
    scores = np.exp(-dE_shifted / tau)  # smaller ΔE -> larger score
    probs = scores / scores.sum()       # normalize to sum to 1

    # 4. Chosen bin = highest probability
    idx = int(np.argmax(probs))   # 0..9
    mst_bin = idx + 1             # 1..10
    best_deltaE = float(dE[idx])
    confidence = float(probs[idx])  # in [0,1]

    return mst_bin, best_deltaE, confidence




In [83]:

#This function computes MST bins for original  images.
def get_original_mst(original_paths, image_processor, model, out_csv="results/mst/original_mst.csv"):
    
    #create results directory if it doesn’t already exist.
    os.makedirs(os.path.dirname(out_csv), exist_ok=True)
    
    # initialize a list to store all results for the CSV.
    rows = []
    
    # Loop over the full file paths of all original images.
    for p in original_paths:
        #load each image as a PIL image and ensure it’s in RGB.
        img = Image.open(p).convert("RGB")
        #Generate a skin mask using your SegFormer model.
        # Convert to boolean mask where True = skin pixel.
        mask = run_inference(img, image_processor, model).astype(bool)
        
        # Compute the MST bin, ΔE to best swatch, and confidence gap.
        mst_bin, dE, conf = assign_mst_bin(img, mask)
        # Skip if no skin found
        if mst_bin is None:
            continue

        # Get hex for this MST bin (1–10 → index 0–9)
        mst_hex = MST_HEX[mst_bin - 1]
        
        # Store results for this image.
        rows.append([p, mst_bin,mst_hex ,dE, conf])
    #Save all results to a CSV with columns for file path, predicted MST bin, ΔE, and confidence.
    pd.DataFrame(rows, columns=["image","mst_bin","mst_hex","deltaE","confidence_gap"]).to_csv(out_csv, index=False)


# a function to compute MST shifts between original and filtered versions of the same images.
def paired_mst_shift(common_names, original_dir, filtered_dir, image_processor, model,
                     out_csv_prefix="results/mst/"):
    os.makedirs(out_csv_prefix, exist_ok=True)
    # Prepare list to store paired results.
    rows = []
    # loop through all filenames that appear in BOTH the original and the filtered folder.
    for name in common_names:
        # construct full paths to both versions of the same image.
        p_orig = os.path.join(original_dir, name)
        p_filt = os.path.join(filtered_dir, name)
        
        # load both images.
        img_o = Image.open(p_orig).convert("RGB")
        img_f = Image.open(p_filt).convert("RGB")

        # compute skin masks for each img
        mask_o = run_inference(img_o, image_processor, model).astype(bool)
        mask_f = run_inference(img_f, image_processor, model).astype(bool)
        

        # assign MST bin for original and filtered images.
        bin_o, dEo, conf_o = assign_mst_bin(img_o, mask_o)
        bin_f, dEf, conf_f = assign_mst_bin(img_f, mask_f)
        
        

        # skip cases where skin could not be detected.
        if bin_o is None or bin_f is None:
            continue
    
        # Look up hex codes from MST_HEX
        hex_o = MST_HEX[bin_o - 1]
        hex_f = MST_HEX[bin_f - 1]
        
        bin_shift = bin_f - bin_o
        # Store filename, original bin, filtered bin, bin shift, ΔE values, and confidence estimates.
        rows.append([
            name,        # image
            bin_o,       # mst_bin_original
            hex_o,       # mst_hex_original
            bin_f,       # mst_bin_filtered
            hex_f,       # mst_hex_filtered
            bin_shift,   # bin_shift
            dEo,         # deltaE_orig
            dEf,         # deltaE_filt
            conf_o,      # conf_gap_orig
            conf_f       # conf_gap_filt
        ])


    # column names for results.
    cols = [
        "image",
        "mst_bin_original", "mst_hex_original",
        "mst_bin_filtered", "mst_hex_filtered",
        "bin_shift",
        "deltaE_orig", "deltaE_filt",
        "conf_gap_orig", "conf_gap_filt"
    ]
    df = pd.DataFrame(rows, columns=cols)
    # Save paired MST results for this filter.
    df.to_csv(os.path.join(out_csv_prefix, os.path.basename(filtered_dir) + "_mst_paired.csv"), index=False)


# get_original_mst(original_images, image_processor, model)
# for filter_name, filter_path in filters.items():
    

In [84]:
# Test filters directories
test_filters = {
    "test_baby_cute_face": test_baby_cute_face,
    "test_baby_girl": test_baby_girl,
    "test_big_city_life": test_big_city_life,
    "test_blochi_gora": test_blochi_gora,
    "test_caramel_macchiato": test_caramel_macchiato,
    "test_cute_baby_face": test_cute_baby_face,
    "test_pretty": test_pretty,
    "test_shiny_foxy": test_shiny_foxy,
    "test_snatched": test_snatched,
}

# Call get_original_mst for test images
# get_original_mst(test_baby_cute_face_images, image_processor, model)

# Process each test filter directory
for filter_name, filter_path in test_filters.items():
    print(f"Processing test filter: {filter_name}")
    paired_mst_shift(test_common_images, original, filter_path, image_processor, model,
                     out_csv_prefix="results/mst/test")

Processing test filter: test_baby_cute_face
Processing test filter: test_baby_girl
Processing test filter: test_big_city_life
Processing test filter: test_blochi_gora
Processing test filter: test_caramel_macchiato
Processing test filter: test_cute_baby_face
Processing test filter: test_pretty
Processing test filter: test_shiny_foxy
Processing test filter: test_snatched


In [85]:
# Call average_ita_diff for each filter directory
filters = {
    "baby_cute_face": baby_cute_face,
    "baby_girl": baby_girl,
    "big_city_life": big_city_life,
    "blochi_gora": blochi_gora,
    "caramel_macchiato": caramel_macchiato,
    "cute_baby_face": cute_baby_face,
    "pretty": pretty,
    "shiny_foxy": shiny_foxy,
    "snatched": snatched,
}

# # Get original
# get_original_mst(original_images, image_processor, model)

for filter_name, filter_path in filters.items():
    print(f"Processing filter: {filter_name}")
    paired_mst_shift(common_images, original, filter_path, image_processor, model,
                     out_csv_prefix="results/mst/")


Processing filter: baby_cute_face
